# N puzzle

Specifications of the problem : [https://en.wikipedia.org/wiki/15_puzzle](https://en.wikipedia.org/wiki/15_puzzle)

In [14]:
from collections import namedtuple
from random import choice
from dataclasses import dataclass, field
from tqdm.auto import tqdm
from icecream import ic
import numpy as np

In [15]:
PUZZLE_DIM = 5

action = namedtuple('Action', ['src', 'dst'])

In [16]:
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions



def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.src], new_state[action.dst] = new_state[action.dst], new_state[action.src]
    return new_state

In [17]:
RANDOMIZE_STEPS = 100_000
SOLUTION = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

state = SOLUTION.copy()
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))

ic(state)
ic(SOLUTION)

Randomizing: 100%|██████████| 100000/100000 [00:00<00:00, 132633.68it/s]
ic| state: array([[14, 18,  8,  7, 11],
                  [10,  4,  6, 20, 17],
                  [16,  5, 13,  9, 24],
                  [22,  0,  1, 12, 15],
                  [ 3, 21, 23,  2, 19]])
ic| SOLUTION: array([[ 1,  2,  3,  4,  5],
                     [ 6,  7,  8,  9, 10],
                     [11, 12, 13, 14, 15],
                     [16, 17, 18, 19, 20],
                     [21, 22, 23, 24,  0]])


array([[ 1,  2,  3,  4,  5],
       [ 6,  7,  8,  9, 10],
       [11, 12, 13, 14, 15],
       [16, 17, 18, 19, 20],
       [21, 22, 23, 24,  0]])

In [18]:
def n_puzzle(state):
    frontier = list()
    while state is not None and not np.array_equal(SOLUTION, state):
        for action in available_actions(state):
            new_state = do_action(state, action)
            frontier.append(new_state)
        state = frontier.pop()
    return state

In [19]:
# final_state = n_puzzle(state)
# ic(final_state)

In [20]:
@dataclass
class Node:
    state: list[list[int]]
    step: int = 0

In [21]:
def n_puzzle_with_explored(state) -> tuple[list[list[int]], int, int] :
    """
    Résolution d'un puzzle en évitant les états redondants.
    Utilise une recherche en profondeur (DFS) avec un ensemble pour mémoriser les états visités.
    """
    frontier: list[Node] = [Node(state=state)]  # La pile des états à explorer
    explored = set()    # Ensemble des états déjà explorés
    cost = 0
    while frontier:
        cost += 1

        # Récupère le prochain état à explorer
        current_node = frontier.pop()

        # Vérifie si la solution est atteinte
        if np.array_equal(SOLUTION, current_node.state):
            return current_node.state, current_node.step, cost

        # Transforme l'état actuel en tuple pour le suivi des explorations
        current_tuple = tuple(map(tuple, current_node.state))

        # Si déjà exploré, passe à l'état suivant
        if current_tuple in explored:
            continue

        # Marque l'état comme exploré
        explored.add(current_tuple)

        # Explore les actions possibles
        for action in available_actions(current_node.state):
            new_state = do_action(current_node.state, action)
            frontier.append(Node(new_state, current_node.step + 1))

    # Si aucune solution n'est trouvée
    return None, None, None

In [22]:
# final_state, quality, cost = n_puzzle_with_explored(state.copy())

# ic(final_state)
# ic(quality)
# ic(cost)

In [23]:
def manhattan_distance(state: np.ndarray) -> int:
    """Calcule la distance de Manhattan entre un état et la solution."""
    total_distance = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value == 0:
                continue  # Ne calcule pas pour la case vide
            target_x, target_y = divmod(value - 1, PUZZLE_DIM)
            total_distance += abs(target_x - x) + abs(target_y - y)
    return total_distance

In [24]:
@dataclass(order=True)
class Node:
    priority: int
    state: list[list[int]] = field(compare=False)
    step: int = field(compare=False)

In [25]:
from heapq import heappop, heappush

def n_puzzle_greedy_best_fit(state):
    """Résolution du N-Puzzle avec Greedy Best Fit."""
    frontier: list[Node] = []  # Utilise un tas (heap) pour prioriser les états
    heappush(frontier, Node(manhattan_distance(state), state, 0))  # (distance, step, state)
    explored = set()
    cost = 0

    while frontier:
        cost += 1
        current_node = heappop(frontier)

        # Vérifie si la solution est atteinte
        if np.array_equal(SOLUTION, current_node.state):
            return current_node.state, current_node.step, cost

        current_tuple = tuple(map(tuple, current_node.state))
        if current_tuple in explored:
            continue

        explored.add(current_tuple)

        # Ajoute les nouveaux états basés sur les actions possibles
        for action in available_actions(current_node.state):
            new_state = do_action(current_node.state, action)
            heappush(frontier, Node(manhattan_distance(new_state), new_state, current_node.step + 1))

    return None, None, cost


In [26]:
final_state, quality, cost = n_puzzle_greedy_best_fit(state.copy())

ic(final_state)
ic(quality)
ic(cost)

ic| final_state: array([[ 1,  2,  3,  4,  5],
                        [ 6,  7,  8,  9, 10],
                        [11, 12, 13, 14, 15],
                        [16, 17, 18, 19, 20],
                        [21, 22, 23, 24,  0]])
ic| quality: 898
ic| cost: 151034


151034